In [1]:
import numpy as np
import pandas as pd
import json
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import random
import datetime

In [11]:
file_dir = "/p/reviewde/data/yelp/"
file_name = "review.json"
full_file_path = file_dir + file_name
print("full_file_path ", full_file_path)

full_file_path  /p/reviewde/data/yelp/review.json


In [12]:
f = open(full_file_path, "r")
l_index = 0

columns = ['rating', 'userid', 'itemid', 'review']
data_list = []

for l in f:
    line = json.loads(l)
    rating = line['stars']
    userid = line['user_id']
    itemid = line['business_id']
#     text = line['text']
    
    if 'text' not in line:
        continue
    review_content = line['text']
    
    sub_data = []
    
    sub_data.append(rating)
    sub_data.append(userid)
    sub_data.append(itemid)
    sub_data.append(review_content)
    
    data_list.append(sub_data)
    
    l_index += 1
    
    if l_index %1e6 == 0:
        print(l_index)
    
f.close()

1000000
2000000
3000000
4000000
5000000
6000000


In [13]:
print("sample num", l_index)
df = pd.DataFrame(data_list)

sample num 6685900


In [14]:
df.columns = columns

In [15]:
sample_num = len(df)
print("sample num in df", sample_num)

sample num in df 6685900


In [16]:
df.head()

,rating,userid,itemid,review
0,1.0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,Total bill for this horrible service? Over $8G...
1,5.0,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,I *adore* Travis at the Hard Rock's new Kelly ...
2,5.0,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,I have to say that this office really has it t...
3,5.0,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,Went in for a lunch. Steak sandwich was delici...
4,1.0,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,Today was my second out of three sessions I ha...


In [17]:
pickle_file = "/p/reviewde/data/yelp/yelp.pickle"
df.to_pickle(pickle_file)

In [18]:
pickle_file = "/p/reviewde/data/yelp/yelp.pickle"
df = pd.read_pickle(pickle_file)

In [4]:
file_dir = "/p/reviewde/data/yelp/"
file_name = "business.json"
full_file_path = file_dir + file_name
print("full_file_path ", full_file_path)

full_file_path  /p/reviewde/data/yelp/business.json


In [5]:
f = open(full_file_path, "r")
l_index = 0

columns = ['categories', 'itemid', 'review_count']
data_list = []

for l in f:
    line = json.loads(l)
    itemid = line['business_id']
    review_count = line['review_count']
    
    if 'categories' not in line:
        continue
    categories = line['categories']
    
    sub_data = []
    
    sub_data.append(categories)
    sub_data.append(itemid)
    sub_data.append(review_count)
    
    data_list.append(sub_data)
    
    l_index += 1
    
    if l_index %1e5 == 0:
        print(l_index)
    
f.close()

100000


In [6]:
item_df = pd.DataFrame(data_list)
item_num = len(item_df)
print("item num", item_num)
item_df.columns = columns

item num 192609


In [7]:
item_df.head()

,categories,itemid,review_count
0,"Golf, Active Life",1SWheh84yJXfytovILXOAQ,5
1,"Specialty Food, Restaurants, Dim Sum, Imported...",QXAEGFB4oINsVuTFxEYKFQ,128
2,"Sushi Bars, Restaurants, Japanese",gnKjwL_1w79qoiV3IC_xQQ,170
3,"Insurance, Financial Services",xvX2CttrVhyG2z1dFg_0xw,3
4,"Plumbing, Shopping, Local Services, Home Servi...",HhyxOkGAM07SRYtlQ4wMFQ,4


In [8]:
categories_list = item_df.categories.tolist()
item_list = item_df.itemid.tolist()
item_num = len(item_list)
print("item num", item_num)
restaurant_id_list = []
for i in range(item_num):
    item_id = item_list[i]
    categories = categories_list[i]
    if not categories:
        continue
#     print("categories", categories)
    
    if 'Restaurants' in categories:
        restaurant_id_list.append(item_id)
    
    if 'restaurants' in categories:
        if item_id in restaurant_id_list:
            continue
        restaurant_id_list.append(item_id)
        
    if 'restaurant' in categories:
        if item_id in restaurant_id_list:
            continue
        restaurant_id_list.append(item_id)
    
    if 'Restaurant' in categories:
        if item_id in restaurant_id_list:
            continue
        restaurant_id_list.append(item_id)
        
print("restaurant item num", len(restaurant_id_list))

item num 192609
restaurant item num 59387


### reserve reviews about restaurant 

In [19]:
review_num = len(df)
print("before filtering", review_num)
df = df[df.itemid.isin(restaurant_id_list)]
review_num = len(df)
print("after filtering", review_num)

before filtering 6685900
after filtering 4201890


In [20]:
min_item_freq_threshold = 10
def remove_infrequent_items(df):
    print(" before sample num ", len(df))
    remove_item_list = []
    item_freq_map = df['itemid'].value_counts()
    for item, freq in item_freq_map.items():
        if freq < min_item_freq_threshold:
            remove_item_list.append(item)
    remove_item_num = len(remove_item_list)
    print("remove item num", remove_item_num, remove_item_num*1.0/len(item_freq_map.keys()))
    print("before filtering item num", df.itemid.nunique())
    df = df[~df.itemid.isin(remove_item_list)]
    print("after filtering item num", df.itemid.nunique())
    print(" after sample num ", len(df))
    
    return df

In [21]:
min_user_freq_threshold = 10
def remove_infrequent_users(df):
    print(" before sample num ", len(df))
    
    user_freq_map = df['userid'].value_counts()
    filter_users = user_freq_map[user_freq_map>=1000].index
    filter_users_num = len(filter_users)

    filter_users_num_total = 0
    print("filter user actions >= 1000", filter_users_num)
    filter_users_num_total += filter_users_num
    df = df[~df.userid.isin(filter_users)]

    filter_users = user_freq_map[user_freq_map<=min_user_freq_threshold].index
    filter_users_num = len(filter_users)
    print("filter user actions <= 10", filter_users_num)
    filter_users_num_total += filter_users_num
    df = df[~df.userid.isin(filter_users)]

    print("filter total user num", filter_users_num_total)
    print("after filtering user num", df.userid.nunique())
    
    print(" after sample num ", len(df))
    
    return df

In [23]:
item_freq_list = df['itemid'].value_counts().tolist()
min_item_freq = min(item_freq_list)

user_freq_list = df['userid'].value_counts().tolist()
min_user_freq = min(user_freq_list)

iter_i = 0

while min_item_freq < min_item_freq_threshold or min_user_freq < min_user_freq_threshold:
    print("=="*10)
    df = remove_infrequent_items(df)
    df = remove_infrequent_users(df)
    
    item_freq_list = df['itemid'].value_counts().tolist()
    min_item_freq = min(item_freq_list)
    print("min_item_freq", min_item_freq)
    
    user_freq_list = df['userid'].value_counts().tolist()
    min_user_freq = min(user_freq_list)
    print("min_user_freq", min_user_freq)
    
    iter_i += 1
    print("iter_i:", iter_i)

 before sample num  4201890
remove item num 17891 0.301261218785256
before filtering item num 59387
after filtering item num 41496
 after sample num  4106524
 before sample num  4106524
filter user actions >= 1000 8
filter user actions <= 5 1069731
filter total user num 1069739
after filtering user num 63721
 after sample num  1847190
min_item_freq 1
min_user_freq 11
iter_i: 1
 before sample num  1847190
remove item num 10945 0.26411042204580004
before filtering item num 41441
after filtering item num 30496
 after sample num  1780407
 before sample num  1780407
filter user actions >= 1000 0
filter user actions <= 5 2711
filter total user num 2711
after filtering user num 61010
 after sample num  1754689
min_item_freq 6
min_user_freq 11
iter_i: 2
 before sample num  1754689
remove item num 510 0.016723504721930744
before filtering item num 30496
after filtering item num 29986
 after sample num  1750253
 before sample num  1750253
filter user actions >= 1000 0
filter user actions <= 5 17

In [24]:
item_freq_list = df['itemid'].value_counts().tolist()
min_item_freq = min(item_freq_list)
print("min_item_freq", min_item_freq)

min_item_freq 10


In [25]:
user_freq_list = df['userid'].value_counts().tolist()
min_user_freq = min(user_freq_list)
print("min_user_freq", min_user_freq)

min_user_freq 11


In [26]:
sample_num = len(df)
print("sample num", sample_num)

sample num 1747948


In [59]:
pickle_file = "/p/reviewde/data/yelp_restaurant_new/yelp.pickle"
df.to_pickle(pickle_file)

In [ ]:
pickle_file = "/p/reviewde/data/yelp_restaurant_new/yelp.pickle"
df = pd.read_pickle(pickle_file)

In [82]:
df.head()

,rating,userid,itemid,review
6,3.0,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,Tracy dessert had a big name in Hong Kong and ...
7,1.0,d6xvYpyzcfbF_AZ8vMB7QA,zvO-PJCpNk4fgAVUnExYAA,This place has gone down hill. Clearly they h...
10,4.0,FIk4lQQu1eTe2EpzQ4xhBA,8mIrX_LrOnAqWsB5JrOojQ,"Like walking back in time, every Saturday morn..."
14,1.0,TpyOT5E16YASd7EWjLQlrw,AakkkTuGZA2KBodKi2_u8A,I cannot believe how things have changed in 3 ...
16,4.0,86J5DwcFk4f4In1Vxe2TvA,NyLYY8q1-H3hfsTwuwLPCg,Great lunch today. Staff was very helpful in a...


In [88]:
item_df[item_df.itemid=="AakkkTuGZA2KBodKi2_u8A"].categories.tolist()

['Asian Fusion, Restaurants, Vietnamese']

In [ ]:
df[df.itemid=="AakkkTuGZA2KBodKi2_u8A"].review.tolist()

In [60]:
from sklearn.feature_extraction.text import CountVectorizer

In [61]:
count_vect = CountVectorizer(max_df=0.05, min_df=20, stop_words='english')

In [62]:
doc_term_matrix = count_vect.fit_transform(df.review.values.astype('U'))

In [65]:
voc = count_vect.vocabulary_

In [69]:
if "service" in voc:
    print("true")

In [70]:
len(voc)

44564

In [79]:
len(count_vect.stop_words_)

279980

In [81]:
len(df.review)*0.05

87397.40000000001

In [71]:
from nltk.corpus import wordnet as wn
import nltk
from nltk import *

In [72]:
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}

In [73]:
noun_voc = []
for word in voc:
    if word in nouns:
        noun_voc.append(word)
print(len(noun_voc))

10631


In [75]:
attr_file = "../data/yelp_restaurant_new/attr.csv"
f = open(attr_file, 'w')
for i in noun_voc:
    f.write(i+'\n')
f.close()

In [76]:
attr_file = "attr.csv"
f = open(attr_file, 'w')
for i in noun_voc:
    f.write(i+'\n')
f.close()

In [32]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [54]:
a = "This is a beautiful chair. \n But I do not like it."

In [55]:
a

'This is a beautiful chair. \n But I do not like it.'

In [56]:
sens = sent_tokenize(a)

In [52]:
for sen in sens:
    words = word_tokenize(sen)
    print("words", words)

words ['This', 'is', 'a', 'beautiful', 'chair', ',', 'But', 'I', 'do', 'not', 'like', 'it', '.']


In [49]:
a = a.replace('\n', ' ')

In [57]:
sens

['This is a beautiful chair.', 'But I do not like it.']

In [58]:
" ".join(sens)

'This is a beautiful chair. But I do not like it.'